## Loading dependencies

In [ ]:
# !pip install langchain ollama python-dotenv
# !pip install -U "langchain-chroma>=0.1.2"
# %pip install -qU langchain-ollama
#!pip install langchain-community
#!pip install jq

In [3]:
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
import chromadb

# Initialize embeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

In [4]:
vector_store = Chroma(
    collection_name="ihec_collection_Orca",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_ihec_db",  
)


## Loading the first dataset

In [5]:
from langchain_community.document_loaders import JSONLoader
# the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["id"] = record.get("id")
    metadata["category"] = record.get("category")
    metadata["question"] = record.get("question")
    metadata["answer"] = record.get("answer")
    return metadata

loader = JSONLoader(
    file_path='./FAQ.json',
    jq_schema=r'.dataset[] | . + {"content": "\(.answer) \(.question) \(.category)"}',
    content_key="content",
    metadata_func=metadata_func
)

documents = loader.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,  
    chunk_overlap=50,  
    separators=["\n\n", "\n", ".", "!", "?", ",", " "]  
)

# 3. Split documents into chunks
split_docs = text_splitter.split_documents(documents)

In [24]:
db = Chroma.from_documents(
    documents=split_docs,  # Use split documents
    embedding=embeddings,
    persist_directory="./chroma_langchain_ihec_db",
    collection_name="ihec_collection_Orca",  # Ensure consistent name
    collection_metadata={"hnsw:space": "cosine"}
)

In [25]:
# 4. Test similarity search
query = "Comment fonctionne le système de transport pour les étudiants ?"
results = db.similarity_search(query, k=2)

print("Top results:")
for doc in results:
    print(f"\n{doc.metadata['answer']}\nMetadata: {doc.metadata}\n---")

Top results:

Nous offrons des navettes gratuites entre le campus et les résidences étudiantes, ainsi que des réductions sur les transports publics.
Metadata: {'answer': 'Nous offrons des navettes gratuites entre le campus et les résidences étudiantes, ainsi que des réductions sur les transports publics.', 'category': 'Vie Étudiante', 'id': 20, 'question': 'Comment fonctionne le système de transport pour les étudiants ?', 'seq_num': 20, 'source': 'C:\\Users\\foura\\Documents\\Orca\\FAQ.json'}
---

Nous offrons des navettes gratuites entre le campus et les résidences étudiantes, ainsi que des réductions sur les transports publics.
Metadata: {'answer': 'Nous offrons des navettes gratuites entre le campus et les résidences étudiantes, ainsi que des réductions sur les transports publics.', 'category': 'Vie Étudiante', 'id': 20, 'question': 'Comment fonctionne le système de transport pour les étudiants ?', 'seq_num': 20, 'source': 'C:\\Users\\foura\\Documents\\Orca\\FAQ.json'}
---


## Loading the second dataset : dataset built from IHEC website

In [23]:
def webpage_metadata_func(record: dict, metadata: dict) -> dict:
    # Ensure metadata values are valid types or provide a default value
    metadata.update({
        "url": record.get("url", "N/A"),  # Default to "N/A" if url is None
        "anchor_text": record.get("anchor_text", "N/A"),  # Default to "N/A"
        "raw_content": record.get("content", "N/A")  # Default to "N/A"
    })
    return metadata

webpage_loader = JSONLoader(
    file_path='ihec_website_dataset.json',
    jq_schema=r'.[] | {"web_content": "\(.anchor_text) \(.content)"}',
    content_key="web_content",
    metadata_func=webpage_metadata_func
)

# Initialize components
web_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=256,
    separators=["\n\n## ", "\n\n", "\n", ". "]
)

def process_dataset(loader, splitter, collection_name, persist_dir):
    docs = loader.load()
    split_docs = splitter.split_documents(docs)
    
    global embeddings  
    if 'embeddings' not in globals():
        raise ValueError("Embeddings variable is not defined.")
    
    return Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_dir,
        collection_metadata={"hnsw:space": "cosine"}
    )

# Create web content vector store
web_db = process_dataset(
    webpage_loader,
    web_splitter,
    collection_name="university_links",
    persist_dir="./chroma_langchain_ihec_db"
)

# Test query
web_query = "Où trouver les emplois ?"
web_results = web_db.similarity_search(web_query, k=2)

print("Liens pertinents:")
for doc in web_results:
    print(f"\n {doc.metadata.get('anchor_text', 'No anchor text')}")
    print(f" {doc.metadata.get('url', 'No URL')}")
    print(f"Extrait: {doc.page_content[:150]}...\n---")

Liens pertinents:

🔗 N/A
🌐 N/A
Extrait: Accédez à de nombreuses opportunités de carrières dans votre secteur grâce à des enseignements variés et de pointe.
Visez une carrière à l’internation...
---

🔗 N/A
🌐 N/A
Extrait: Accédez à de nombreuses opportunités de carrières dans votre secteur grâce à des enseignements variés et de pointe.
Visez une carrière à l’internation...
---

🔗 N/A
🌐 N/A
Extrait: Pour atteindre l’objectif de l’employabilité, une mise en relation de l’étudiant en quête, de stage, d’emplois ou d'incubation et les entreprises en q...
---


In [ ]:
# reloaded_vector_store = Chroma(
#     collection_name="ihec_collection",
#     embedding_function=embeddings,
#     persist_directory="./chroma_langchain_db",  # Match original path
# )

## Generation

In [27]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.2,
    max_tokens=200
)

In [35]:
from langchain_core.messages import HumanMessage, SystemMessage
# Prompt
rag_prompt = """You are an assistant for question-answering task.

Your role is to guide and explain  students 

Here is the context to use to answer the question:

{context} 

Think carefully about the above context. 

Now, review the user question:

{question}

Provide an answer to this questions using only the above context. 

Use three sentences maximum and keep the answer concise.

Answer:"""

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Test
question= "Comment accéder aux laboratoires de recherche ?"

# Perform similarity searches
results = db.similarity_search(query, k=2)
web_results = web_db.similarity_search(web_query, k=3)

# Extract `page_content` from both sets of results
combined_content = " ".join([doc.page_content for doc in results + web_results])


rag_prompt_formatted = rag_prompt.format(context=combined_content, question=question)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
print(generation.content)

Je suis désolé, mais le contexte fourni ne mentionne pas les laboratoires de recherche. Il semble que le contexte se concentre sur l'offre d'éducation et les opportunités de carrière pour les étudiants, ainsi que les services de transport gratuits entre le campus et les résidences étudiantes.

Cependant, si vous cherchez à accéder aux laboratoires de recherche, je suppose qu'il faudrait contacter directement l'université ou la faculté concernée pour obtenir plus d'informations sur les procédures d'accès.
